# First Attempt implementing FC neural network in PyTorch

In [1]:
import torch # God please write code for me

## Problem Description

The aim of the model is to estimate the [IMDB](https://www.imdb.com) score of the a movie, based on:

- Name
- Release Date
- Production Budget
- Genre
- Worldwide Gross
- Run Time

We then try to identify this relationship.

$$\left.
\begin{array}{l}
\text{Film Name} \\
\text{Release Date} \\
\text{Production Budget} \\
\text{Genre} \\
\text{Worldwide Gross} \\
\text{Run Time}
\end{array}
\right\}\Rightarrow\text{IMDB Rating}$$

## Data Source

The data source comes from [here](https://vega.github.io/editor/data/movies.json), licensed under MIT License.

## Preprocessing

For preprocessing, I used a combination of Lua and Python.

### Filter

The original JSON file has more than 3000 films, but many of which has missing information.

The `preprocess.lua` **filters out all entries without any one of the following parameters**:

- Film Name (`Title`)
- Release Date (`Release Date`)
- Production Budget (`Production Budget`)
- Genre (`Main Genre`)
- Worldwide Gross (`Worldwide Gross`)
- Run Time (`Running Time min`)
- IMDB Rating (`IMBD Rating`)

After filtering, **1141 films** survived with **12 different genres**:

```json
["Action","Black Comedy","Comedy","Adventure","Drama","Romantic Comedy","Horror","Thriller/Suspense","Musical","Documentary","Western","Concert/Performance"]
```

The list of movies that survived is stored in `movies_processed.json`.

The list of genres are stored in `genres.json`.

### Encoding

Before feeding the data into the FC neural network, we first need to find a way to encode the data into a list of numbers.

In [2]:
import json, os

with open("genres.json") as f:
    Genres = json.loads(f.read())
with open("movies_processed.json") as f:
    Movies = json.loads(f.read())
print("%d movies found with %d genres." % (len(Movies), len(Genres)))

1140 movies found with 12 genres.


### Input and Output

We'll input all information of a movie into a FC network. But before doing so, proper encoding is necessary.

#### Encoding

In [3]:
import numpy as np
EncodedInput = []

# Titles: stats
maxLen = -1
maxAscii = -1
minAscii = np.Infinity
for i in Movies:
    t = str(i['Title'])
    if len(t) > maxLen:
        maxLen = len(t)
    for char in t:
        if ord(char) < minAscii:
            minAscii = ord(char)
            print("MIN @", char, "({0})".format(ord(char)))
        if ord(char) > maxAscii:
            maxAscii = ord(char)
            print("MAX @", char, "({0})".format(ord(char)))
print('maxLen = {0}\nmaxAscii = {1}\nminAscii = {2}'.format(maxLen, maxAscii, minAscii))

MIN @ B (66)
MAX @ B (66)
MAX @ r (114)
MIN @   (32)
MAX @ w (119)
MAX @ z (122)
MAX @ È (200)
MAX @ ‡ (8225)
maxLen = 58
maxAscii = 8225
minAscii = 32


In the above snippet we found that the maximum length for a title is 58. Several strange symbols are found in addition to regular `(space)` to `z` in ASCII representations. To encode the title, any character with `ord(char) > 122 or ord(char) < 32` will be **clamped** into the range.

The following code implements such encoding.

In [4]:
# Clamping function
def clamp(val, valMin, valMax):
    if val > valMax:
        return valMax, True
    elif val < valMin:
        return valMin, True
    else:
        return val, False

# ASCII encode
TitleEncoded = list(list(clamp(ord(char), 32, 122)[0] for char in str(i["Title"])) for i in Movies)
# Extending to maxLen
TitleEncoded = list(i + [0] * (maxLen - len(i)) for i in TitleEncoded)

# Confirmation
print(Movies[0]["Title"], TitleEncoded[0])

# Add to EncodedInput
EncodedInput.append(TitleEncoded)

Broken Arrow [66, 114, 111, 107, 101, 110, 32, 65, 114, 114, 111, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


The title in its ASCII form will be given as input to the neural network in **an array of 58 integers** ranging from 32 to 122.

---

Before moving on, define the following normalization function first. It maps an array of data into an array of float between `normMin` and `normMax`.

In [5]:
def normalize(arr, normMin, normMax):
    inMin, inMax = min(arr), max(arr)
    return list(map(lambda val: (val - inMin) / (inMax - inMin) * (normMax - normMin) + normMin, arr))

Then we'll move on to release dates.

In [6]:
import time

# Example: Jul 25 2008
Timestamps = list(time.mktime(time.strptime(i["Release Date"], "%b %d %Y")) / 100 for i in Movies)
print("Release Date Timestamps ranges from", min(Timestamps), "to", max(Timestamps))

Release Date Timestamps ranges from 3296736.0 to 22074912.0


In the above snippet we found that the time starts with a big integer and has a rather big range. To normalize the time (foundamentally logical because time is consecutive), we set a linear normalization function to clamp all ranges into $[0,\,1]$.

In [7]:
NormalizedTimestamps = normalize(Timestamps, 0, 1)
# Add to EncodedInput
EncodedInput.append(list([i] for i in NormalizedTimestamps))

The timestamps takes **1** float value when given as input to the neural network.

In [8]:
NormalizedProductionBudgets = normalize(list(i["Production Budget"] for i in Movies), 0, 1)
print(NormalizedProductionBudgets[0], Movies[0]["Production Budget"])
EncodedInput.append(list([i] for i in NormalizedProductionBudgets))

0.21664838846239745 65000000


In [9]:
def getGenreId(str):
    return Genres.index(str)
GenreId = list(map(getGenreId, list(i["Major Genre"] for i in Movies)))
print(GenreId[0], Movies[0]["Major Genre"])
EncodedInput.append(list([i] for i in GenreId))

0 Action


In [10]:
NormalizedWorldwideGross = normalize(list(i["Worldwide Gross"] for i in Movies), 0, 1)
print(NormalizedWorldwideGross[0], Movies[0]["Worldwide Gross"])
EncodedInput.append(list([i] for i in NormalizedWorldwideGross))

0.08049054258816266 148345997


In [11]:
EncodedInput.append(list([i["Running Time min"]] for i in Movies))

Before finishing, let's check the final results in `EncodedInput`:

In [12]:
print(len(EncodedInput))
print(list(i[0] for i in EncodedInput), Movies[0])

6
[[66, 114, 111, 107, 101, 110, 32, 65, 114, 114, 111, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.26313610011962824], [0.21664838846239745], [0], [0.08049054258816266], [108]] {'IMDB Votes': 33584, 'Production Budget': 65000000, 'IMDB Rating': 5.8, 'MPAA Rating': 'R', 'Rotten Tomatoes Rating': 55, 'Distributor': '20th Century Fox', 'Director': 'John Woo', 'Creative Type': 'Contemporary Fiction', 'US Gross': 70645997, 'Major Genre': 'Action', 'Worldwide Gross': 148345997, 'Source': 'Original Screenplay', 'Release Date': 'Feb 09 1996', 'Running Time min': 108, 'Title': 'Broken Arrow'}


Now we can generate the encoded input for all movies.

In [13]:
from functools import reduce

def concat(list1, list2):
    list1.extend(list2)
    return list1

EncodedInput = list(reduce(concat, list(i[j] for i in EncodedInput)) for j in range(0, len(Movies)))

In [16]:
print(len(EncodedInput))
print(EncodedInput[0], Movies[0])

1140
[66, 114, 111, 107, 101, 110, 32, 65, 114, 114, 111, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.26313610011962824, 0.21664838846239745, 0, 0.08049054258816266, 108] {'IMDB Votes': 33584, 'Production Budget': 65000000, 'IMDB Rating': 5.8, 'MPAA Rating': 'R', 'Rotten Tomatoes Rating': 55, 'Distributor': '20th Century Fox', 'Director': 'John Woo', 'Creative Type': 'Contemporary Fiction', 'US Gross': 70645997, 'Major Genre': 'Action', 'Worldwide Gross': 148345997, 'Source': 'Original Screenplay', 'Release Date': 'Feb 09 1996', 'Running Time min': 108, 'Title': 'Broken Arrow'}


### Groundtruths

The groundtruth in this case is the IMDB rating of the 1140 movies. Simply compile the groundtruths based on the movies.


In [18]:
GroundTruth = list(i["IMDB Rating"] for i in Movies)

## Constructing Neural Network

Now it's time to make a neural network using basic PyTorch functions!